# Protein structure superposing



## 3) **Superposing with Python**

Gemmi is an efficient parser for the `mmCIF` file format. We will be using it here to parse in structures from the PDBe we have saved locally. Once loaded into the script, we will explore different protein structure superposition tools. 

In [ ]:
import subprocess
import pathlib
import gemmi
import Bio

### 3.1) Loading and saving structure files

Structure files, typically in PDB or mmCIF formats, can be parsed into Python scripts using the built-in read-write functions, such as `open()` and `write()`. However, tools have been developed to make this process quick and easy. We will be focusing on the file parsing module Gemmi as it is relatively quick and has good support for the modern mmCIF file format. Another set of functions for parsing structure files into Python is included in the popular Biopython library, which tends to focus on files in the PDB format. 


#### **Gemmi**

[Gemmi](https://gemmi.readthedocs.io/en/latest/) is a modern Python and C++ library containing many tools for performing common tasks in structural biology. It is well suited to handling mmCIF (PDBx/mmCIF) file formats, which are organised into blocks and loops of data.

To load a mmCIF file:

In [ ]:
from gemmi import cif

# Location of saved structure file
path_mmcifs = "examples_mmcif/"
path_4pqe = str( path_mmcifs + "4pqe.cif" )

# Load file into program
mmcif_4pqe = cif.read_file( path_4pqe ).sole_block()

To extract information from the parsed mmCIF file, we can use the `find()` method, where we provide the blocks and loops of data we want to access. For a complete list of all data blocks and loops, refer to the [file format documentation](https://mmcif.wwpdb.org/dictionaries/mmcif_pdbx_v50.dic/Categories/index.html) provided by the [wwPDB](http://www.wwpdb.org/). 

As an introductory example, the code below will extract the atom type, atom label, atom coordinates, occupancy and chain ID (author stated).

In [3]:
import numpy as np
import pandas as pd

# Extract data
mmcif_table =  mmcif_4pqe.find(
    "_atom_site.",
    [
        "group_PDB",
        "label_atom_id",
        "Cartn_x",
        "Cartn_y",
        "Cartn_z",
        "occupancy",
        "auth_asym_id"
    ],
    )

# Convert to Numpy array
# ... not strictly necessary but improves runtime performance and data handling
mmcif_array = np.asarray(mmcif_table)

# Convert to Pandas DataFrame
# ... especially useful for data handling and selecting rows
mmcif_df = pd.DataFrame(
    mmcif_table,
    columns=[
        "group_PDB",
        "label_atom_id",
        "Cartn_x",
        "Cartn_y",
        "Cartn_z",
        "occupancy",
        "auth_asym_id"
        ]
    )

# Extract only the CA atoms
ca_atoms = mmcif_df.loc[mmcif_df["label_atom_id"] == "CA"]
print(ca_atoms)

     group_PDB label_atom_id  Cartn_x Cartn_y Cartn_z occupancy auth_asym_id
1         ATOM            CA    3.743   4.895   5.425      1.00            A
6         ATOM            CA    3.011   3.527   8.949      1.00            A
11        ATOM            CA    0.377   6.211   9.189      1.00            A
20        ATOM            CA   -1.533   4.782   6.187      1.00            A
28        ATOM            CA   -2.913   1.673   8.092      1.00            A
...        ...           ...      ...     ...     ...       ...          ...
4050      ATOM            CA  -47.653  35.142   4.314      1.00            A
4057      ATOM            CA  -50.813  37.219   5.212      1.00            A
4066      ATOM            CA  -51.627  37.334   1.556      1.00            A
4074      ATOM            CA  -51.263  33.778   0.342      1.00            A
4082      ATOM            CA  -53.597  33.392   3.272      1.00            A

[528 rows x 7 columns]


In [4]:
# Load example structure into script
model_original = gemmi.read_structure('./examples_mmcif/1lzh_updated.cif') # PDB format accepted
print(model_original[0])

model = model_original[0]

# Assign static chain
static = model['A'].get_polymer()
# Assign mobile chains
mobile = model['B'].get_polymer()
ptype = static.check_polymer_type()     # Useful when not already known

# Perform QCP superposition
superposed = gemmi.calculate_superposition(
    static,               # Structure 1
    mobile,               # Structure 2
    ptype,                  # The type of macromolecule
    gemmi.SupSelect.CaP     # Select only CAs
    )

# RMSD from superposition
print("RMSD = ", superposed.rmsd)

# Some useful information on the superposition of the transformation
superposed_matrix = superposed.transform.mat
superposed_vector = superposed.transform.vec

superposed_matrix = np.asarray(superposed_matrix)   # Convert to ndarry to make visible
superposed_vector = np.asarray(superposed_vector)   # Convert to ndarry to make visible

print(superposed_matrix)
print(superposed_vector)

res_num = 50
print(model['A'].get_polymer()[res_num].sole_atom('CA'))
print(model['B'].get_polymer()[res_num].sole_atom('CA'))

# Transforming the mobile polymer based on superposition transformation matrix
mobile.transform_pos_and_adp(superposed.transform)

# Save suposition
model_original.write_pdb("example_qcp_superposition.pdb", 
          seqres_records= True,
          ssbond_records= True,
          link_records= True,
          cispep_records= True,
          ter_records = True,
          numbered_ter = True,
          ter_ignores_type = True,
          use_linkr= True
          )

model_original.make_mmcif_document().write_file('example_qcp_superposition.cif')

<gemmi.Model 1 with 2 chain(s)>
RMSD =  0.005110694766999929
[[ 0.97570792 -0.20759995  0.06997372]
 [ 0.21560375  0.96658406 -0.13867325]
 [-0.03884692  0.15039119  0.98786305]]
<gemmi.Vec3(-14.1959, 0.730062, -30.523)>
<gemmi.Atom CA at (6.8, 5.5, -16.4)>
<gemmi.Atom CA at (21.0, 2.4, 14.8)>


### 3.2) Command line tools via Python

Some tools are often only available as command line applications, meaning they lack a dedicated Python (or any other programming language) API. This prevents us from using the code for these tools directly in our code. However, we can still use these applications through the command line using the `subprocess` module. You will sometimes see the `os` module used for functionally similar purposes. Which you decide to use is often a matter of preference. We will be considering the `subprocess` module as it is rich in features relative to `os`. 

`subprocess` allows us to send commands to the terminal through Python. Take a look at the general example below:



In [ ]:
import subprocess

# Define the command to run in the terminal
path = '.'
command = [f"ls -ltr {path}"]       # ls -ltr .

# Ensure results are formatted as strings
args = {
    "shell" : True, 
    "encoding" : "utf-8"
    }

# Send the command to the terminal for execution
results = subprocess.check_output(command, **args).splitlines()

# Display the results, line-by-line, from the command
for line in results:
    print(line)

total 400
-rw-r--r--   1 mbage  staff    5715  3 May 09:53 superposition_1_web.ipynb
drwxr-xr-x   9 mbage  staff     288  3 May 14:08 figures
-rw-r--r--   1 mbage  staff   13438  3 May 14:09 superposition_2_local.ipynb
-rw-r--r--   1 mbage  staff  121297  3 May 14:31 superposition_3_python.ipynb
-rw-r--r--   1 mbage  staff     931  3 May 14:31 gesamt_via_python.py
drwxr-xr-x  10 mbage  staff     320  3 May 14:32 examples_mmcif
drwxr-xr-x  11 mbage  staff     352  3 May 14:34 examples_pdb
-rw-r--r--   1 mbage  staff   25434  3 May 14:34 example_qcp_superposition.pdb
-rw-r--r--   1 mbage  staff   23643  3 May 14:34 example_qcp_superposition.cif


Providing you are in the same directory in which this notebook is running, executing the command `ls -ltr .` will give you exactly the same result. 

To simply run a command in the terminal without saving the results, use:

In [ ]:
import os

# Define the command to run in the terminal
path = '.'
command = f"ls -ltr {path}"       # ls -ltr .

# Execute the command in the terminal. Results not saved
results = os.system(command)

total 400
-rw-r--r--   1 mbage  staff    5715  3 May 09:53 superposition_1_web.ipynb
drwxr-xr-x   9 mbage  staff     288  3 May 14:08 figures
-rw-r--r--   1 mbage  staff   13438  3 May 14:09 superposition_2_local.ipynb
-rw-r--r--   1 mbage  staff  121297  3 May 14:31 superposition_3_python.ipynb
-rw-r--r--   1 mbage  staff     931  3 May 14:31 gesamt_via_python.py
drwxr-xr-x  10 mbage  staff     320  3 May 14:32 examples_mmcif
drwxr-xr-x  11 mbage  staff     352  3 May 14:34 examples_pdb
-rw-r--r--   1 mbage  staff   25434  3 May 14:34 example_qcp_superposition.pdb
-rw-r--r--   1 mbage  staff   23643  3 May 14:34 example_qcp_superposition.cif


_NB_: `ls` lists all of the files and subdirectories in a specified folder, `-ltr` are flags that modify the command's behaviour, and `.` tells the command to work on _this_ directory. `-ltr` is equivalent to `-l -t -r`, which are flags to tell `ls` to provide extended file/directory information, order the results by date modified, and reverse this order to place most recent files at the bottom, respecitvely. 

We can, therefore, implement `subprocess` commands in Python scripts to run superposition applications. Of course, this package is not exclusive to superposition and can be used to run any command line application automatically.

The code below will serially superpose the structures in the `examples_mmcif` directory, using GESAMT.  

In [8]:
import subprocess

# Define the command for running superposition application
gesamt = "gesamt"

# Strings that will not change in script
path_structures = "./examples_mmcif/"
static_struct = "4pqe.cif"
path_static = f"{path_structures}{static_struct}"

# Loop over the structures for superposition
for mobile_struct in ["6emi.cif", "6cqz.cif" "4ey7.cif", "4ey8.cif"]:

    # Path to mobile structure in superposition
    path_mobile = f"{path_structures}{mobile_struct}"

    # Out file names
    save_file_gesamt = f"gesamt_{mobile_struct[:4]}_to_{static_struct[:4]}.pdb"

    # GESAMT command ready
    command_gesamt = f"{gesamt} {path_static} {path_mobile} -o {save_file_gesamt}"

    # Execute GESAMT command
    os.system(command_gesamt)


 GESAMT: General Efficient Structural Alignment of Macromolecular Targets
 ------------------------------------------------------------------------
 Version 1.18 of 24-Mar-2022, built with MMDB v.2.0.22
 
 ###############################################################
 ###############################################################
 ###############################################################
 ### CCP4 8.0.011: Gesamt           version 8.0.011 :         ##
 ###############################################################
 User: mbage  Run date:  3/ 5/2023 Run time: 14:35:27 


 Please reference: Collaborative Computational Project, Number 4. 2011.
 "Overview of the CCP4 suite and current developments". Acta Cryst. D67, 235-242.
 as well as any specific reference in the program write-up.

$TEXT:Reference: $$Please cite$$
E. Krissinel (2012). Enhanced fold recognition using efficient
short fragment clustering. J. Mol. Biochem. 1(2) 76-85.
$$
<!--SUMMARY_BEGIN-->


 ... reading FIXED 

|H- A:ALA 420 | <**0.16**> |H- A:ALA 420 |
|H+ A:GLN 421 | <**0.32**> |H+ A:GLN 421 |
|S- A:GLY 422 | <**0.46**> |S- A:GLY 422 |
| - A:ALA 423 | <**0.38**> | - A:ALA 423 |
|S+ A:ARG 424 | <**0.35**> |S+ A:ARG 424 |
|S- A:VAL 425 | <**0.33**> |S- A:VAL 425 |
|S. A:TYR 426 | <**0.20**> |S. A:TYR 426 |
|S- A:ALA 427 | <**0.28**> |S- A:ALA 427 |
|S. A:TYR 428 | <**0.22**> |S. A:TYR 428 |
|S- A:VAL 429 | <**0.18**> |S- A:VAL 429 |
|S- A:PHE 430 | <**0.14**> |S- A:PHE 430 |
| + A:GLU 431 | <**0.27**> | + A:GLU 431 |
| + A:HIS 432 | <**0.36**> | + A:HIS 432 |
| + A:ARG 433 | <**0.26**> | + A:ARG 433 |
| - A:ALA 434 | <**0.22**> | - A:ALA 434 |
| . A:SER 435 | <**0.16**> | . A:SER 435 |
| . A:THR 436 | <**0.19**> | . A:THR 436 |
| - A:LEU 437 | <**0.33**> | - A:LEU 437 |
| . A:SER 438 | <**0.48**> | . A:SER 438 |
| . A:TRP 439 | <**0.37**> | . A:TRP 439 |
| + A:PRO 440 | <**0.46**> | + A:PRO 440 |
| - A:LEU 441 | <**0.38**> | - A:LEU 441 |
| . A:TRP 442 | <**0.37**> | . A:TRP 442 |
| - A:MET 4

If you receive a `command not found` error message, instead run the `gesamt_via_python.py` script, which contains exactly the same code. 

### **Practice exercise 3:** Serial superposition

Using your results collected from the initial search of 5igh against one of your chosen databases from exercise 1), write a Python script to superpose the models using either GESAMT or SSM. It might be useful to delegate some of the work across the group, allowing you to compare the SSM and GESAMT results to see if they agree. 

Once superposed, you can load your results into PyMol or Mol* and begin to identify common ligand-binding features between structures. 